<a href="https://colab.research.google.com/github/luuloi/GWAS_Introduction_2023/blob/main/06_PolygenicRiskScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP ENVIRONMENT

In [52]:
!pip install rpy2==3.5.1
!pip install -q condacolab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 22.6 MB/s eta 0:00:00
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=247992 sha256=6de45904b264a768418007790b32184432135d92b617ca30ae6ba41b5bec0024
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.5.13
    Uninstalling rpy2-3.5.13:
      Successfully uninstalled rpy2-3.5.13


In [53]:
import rpy2

In [46]:
# Ignore rpy2's warnings
import warnings
from rpy2.rinterface import RRuntimeWarning

warnings.filterwarnings("ignore", category=RRuntimeWarning)

In [47]:
# Initialize conda

import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [48]:
from rpy2.robjects import conversion, default_converter
with conversion.localconverter(default_converter):
    %load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Set up R environment by installing required packages

In [34]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


# INSTALL PLINK PACKAGE

In [49]:
# Install plink

!conda install -c bioconda plink

Solving environment: - \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
  current version: 23.1.0
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2



# All requested packages already installed.



# PREPARE DATA

In [ ]:
%%bash
mkdir data
mkdir -p data/base
mkdir -p data/target

mkdir: cannot create directory ‘data’: File exists


Check the data:

In [ ]:
%%bash
zcat data/base/Height.gwas.txt.gz | head

CHR	BP	SNP	A1	A2	N	SE	P	OR	INFO	MAF
1	756604	rs3131962	A	G	388028	0.00301666	0.483171	0.997886915712657	0.890557941364774	0.369389592764921
1	768448	rs12562034	A	G	388028	0.00329472	0.834808	1.00068731609353	0.895893511351165	0.336845754096289
1	779322	rs4040617	G	A	388028	0.00303344	0.42897	0.997603556067569	0.897508290615237	0.377368010940814
1	801536	rs79373928	G	T	388028	0.00841324	0.808999	1.00203569922793	0.908962856432993	0.483212245374095
1	808631	rs11240779	G	A	388028	0.00242821	0.590265	1.00130832511154	0.893212523690488	0.450409558999587
1	809876	rs57181708	G	A	388028	0.00336785	0.71475	1.00123165786833	0.923557624081969	0.499743932656759
1	835499	rs4422948	G	A	388028	0.0023758	0.710884	0.999119752645202	0.906437735120596	0.481016005816168
1	838555	rs4970383	A	C	388028	0.00235773	0.150993	0.996619945289758	0.907716506801574	0.327164029672754
1	840753	rs4970382	C	T	388028	0.00207377	0.199967	0.99734567895614	0.914602590137255	0.498936220426316


In [51]:
%%bash
python

In [50]:
%%R
getwd()

NotImplementedError: ignored

In [ ]:
%%R
df <- read.csv("sample_data/california_housing_test.csv")

In [ ]:
%%R
head(df)

  longitude latitude housing_median_age total_rooms total_bedrooms population
1   -122.05    37.37                 27        3885            661       1537
2   -118.30    34.26                 43        1510            310        809
3   -117.81    33.78                 27        3589            507       1484
4   -118.36    33.82                 28          67             15         49
5   -119.67    36.33                 19        1241            244        850
6   -119.56    36.51                 37        1018            213        663
  households median_income median_house_value
1        606        6.6085             344700
2        277        3.5990             176500
3        495        5.7934             270500
4         11        6.1359             330000
5        237        2.9375              81700
6        204        1.6635              67000
